In [1]:
#importar biliotecas
import rasterio
import numpy as np
import joblib
import pandas as pd
from tqdm import tqdm

In [48]:
#  Caminhos 
path_raster = "raster_empilhado.tif"
path_modelo = "modelo_inundacao_RF.pkl"
path_scaler = "scaler_inundacao.pkl" 
output_map = "mapa_suscetibilidade_inundacao.tif"


In [49]:
# Carregar modelo treinado E O SCALER 
print("\nCarregando modelo e scaler...")
rf = joblib.load(path_modelo)
scaler = joblib.load(path_scaler)  
print(" Modelo carregado: modelo_inundacao_RF.pkl")
print(" Scaler carregado: scaler_inundacao.pkl")


Carregando modelo e scaler...
 Modelo carregado: modelo_inundacao_RF.pkl
 Scaler carregado: scaler_inundacao.pkl


In [50]:
# Abrir raster empilhado
print("\nLendo raster empilhado...")
with rasterio.open(path_raster) as src:
    data = src.read().astype("float32")  
    profile = src.profile


Lendo raster empilhado...


In [51]:
# Converter raster em tabela (pixels x bandas) 
bands, rows, cols = data.shape
print(f" Raster lido: {bands} bandas, {rows}x{cols} pixels")
print(f" Total de pixels: {rows * cols:,}")

 Raster lido: 15 bandas, 1798x1079 pixels
 Total de pixels: 1,940,042


In [52]:
# Reorganizar dados para (n_pixels, n_bandas)
X = data.reshape(bands, -1).T

In [53]:
# === Limpar e tratar NaN ===
print("\nProcessando pixels válidos...")
mask_nan = np.any(np.isnan(X), axis=1)
X_valid = X[~mask_nan]

print(f"Pixels válidos: {len(X_valid):,} / {len(X):,}")
print(f"Pixels com NaN removidos: {np.sum(mask_nan):,}")



Processando pixels válidos...
Pixels válidos: 675,192 / 1,940,042
Pixels com NaN removidos: 1,264,850


In [54]:
#  Converter para DataFrame
print("\nPreparando dados para predição...")
X_df = pd.DataFrame(X_valid, columns=[
    # Índices espectrais
    'NDVI', 'NDWI', 'NDBI', 'MNDWI', 'SAVI', 'BSI',

    # Variáveis topográficas e hidrológicas (ordem real no raster)
    'TWI',
    'Altitude', 'Curvatura_H', 'Curvatura_V', 'Declividade',
    'Divisores', 'Forma_terreno', 'Orientacao', 'Relevo'
])


Preparando dados para predição...


In [55]:
# aplicar a normalizacao com o mesmo scaler do treino
print("Aplicando normalização (StandardScaler)")
X_df_scaled = scaler.transform(X_df)
X_df = pd.DataFrame(X_df_scaled, columns=X_df.columns)
print(" Dados normalizados")

Aplicando normalização (StandardScaler)
 Dados normalizados


In [56]:
# Predizer em lotes 
batch_size = 50000
y_pred_prob = np.zeros(len(X_df), dtype='float32')

for i in tqdm(range(0, len(X_df), batch_size), desc="   Processando"):
    end_idx = min(i + batch_size, len(X_df))
    y_pred_prob[i:end_idx] = rf.predict_proba(X_df.iloc[i:end_idx])[:, 1]

print(f" Previsões concluídas")
print(f" Probabilidade média: {y_pred_prob.mean():.4f}")
print(f" Probabilidade mín: {y_pred_prob.min():.4f}")
print(f" Probabilidade máx: {y_pred_prob.max():.4f}")

   Processando: 100%|██████████| 14/14 [00:03<00:00,  4.16it/s]

 Previsões concluídas
 Probabilidade média: 0.3986
 Probabilidade mín: 0.0326
 Probabilidade máx: 0.9805


In [57]:
# Criar array final com NaN onde faltou dado 
print("\n Reconstruindo imagem espacial...")
pred_full = np.full(X.shape[0], np.nan, dtype="float32")
pred_full[~mask_nan] = y_pred_prob


 Reconstruindo imagem espacial...


In [58]:
# Voltar ao formato espacial (rows, cols)
mapa_pred = pred_full.reshape(rows, cols)

In [59]:
# Atualizar perfil para 1 banda 
profile.update(count=1, dtype="float32", nodata=np.nan)

In [60]:
# Salvar GeoTIFF
print("\nSalvando mapa...")
with rasterio.open(output_map, "w", **profile) as dst:
    dst.write(mapa_pred, 1)


Salvando mapa...


In [61]:
# Estatísticas finais
print("\n" + "="*60)
print("ESTATÍSTICAS DO MAPA GERADO")
print("="*60)
print(f"Dimensões: {rows} x {cols} pixels")
print(f"Pixels classificados: {(~np.isnan(mapa_pred)).sum():,}")
print(f"Pixels com alta suscetibilidade (>0.7): {(mapa_pred > 0.7).sum():,}")
print(f"Pixels com média suscetibilidade (0.3-0.7): {((mapa_pred >= 0.3) & (mapa_pred <= 0.7)).sum():,}")
print(f"Pixels com baixa suscetibilidade (<0.3): {(mapa_pred < 0.3).sum():,}")
print("="*60)
print("✓ Processamento concluído com sucesso!")
print("="*60)


ESTATÍSTICAS DO MAPA GERADO
Dimensões: 1798 x 1079 pixels
Pixels classificados: 675,192
Pixels com alta suscetibilidade (>0.7): 102,258
Pixels com média suscetibilidade (0.3-0.7): 275,568
Pixels com baixa suscetibilidade (<0.3): 297,366
✓ Processamento concluído com sucesso!


In [ ]:
import rasterio
import numpy as np
import joblib
import pandas as pd
from tqdm import tqdm

print("="*60)
print("GERAÇÃO DO MAPA DE SUSCETIBILIDADE À INUNDAÇÃO")
print("="*60)

# === Caminhos ===
path_raster = "raster_empilhado.tif"
path_modelo = "modelo_inundacao_RF.pkl"
path_scaler = "scaler_inundacao.pkl"  
output_map = "mapa_suscetibilidade_inundacao.tif"

# === Carregar modelo treinado E O SCALER ===
print("\n Carregando modelo e scaler...")
rf = joblib.load(path_modelo)
scaler = joblib.load(path_scaler)  
print("   ✓ Modelo carregado: modelo_inundacao_RF.pkl")
print("   ✓ Scaler carregado: scaler_inundacao.pkl")

# === Abrir raster empilhado ===
print("\n Lendo raster empilhado...")
with rasterio.open(path_raster) as src:
    data = src.read().astype("float32")  # shape: (bands, rows, cols)
    profile = src.profile

# === Converter raster em tabela (pixels x bandas) ===
bands, rows, cols = data.shape
print(f"  Raster lido: {bands} bandas, {rows}x{cols} pixels")
print(f"  Total de pixels: {rows * cols:,}")

# Reorganizar dados para (n_pixels, n_bandas)
X = data.reshape(bands, -1).T

# === Limpar e tratar NaN ===
print("\nProcessando pixels válidos...")
mask_nan = np.any(np.isnan(X), axis=1)
X_valid = X[~mask_nan]

print(f"   ✓ Pixels válidos: {len(X_valid):,} / {len(X):,}")
print(f"   ✓ Pixels com NaN removidos: {np.sum(mask_nan):,}")

# === Converter para DataFrame ===
print("\n Preparando dados para predição...")
X_df = pd.DataFrame(X_valid, columns=[
    'NDVI', 'NDWI', 'NDBI', 'MNDWI', 'SAVI', 'BSI',
    'Altitude', 'Curvatura_H', 'Curvatura_V', 'Declividade',
    'Forma_Terreno', 'Relevo', 'Orientacao', 'Divisores'
])

#  APLICAR A NORMALIZAÇÃO COM O MESMO SCALER DO TREINO
print("  Aplicando normalização (StandardScaler)...")
X_df_scaled = scaler.transform(X_df)  # 🔥 CRÍTICO: usar transform, não fit_transform
X_df = pd.DataFrame(X_df_scaled, columns=X_df.columns)
print("   ✓ Dados normalizados")

# === Prever probabilidade de inundação ===
print("\nGerando previsões...")
print("   (Isso pode levar alguns minutos...)")

# Predizer em lotes para economizar memória (opcional, mas recomendado)
batch_size = 50000
y_pred_prob = np.zeros(len(X_df), dtype='float32')

for i in tqdm(range(0, len(X_df), batch_size), desc="   Processando"):
    end_idx = min(i + batch_size, len(X_df))
    y_pred_prob[i:end_idx] = rf.predict_proba(X_df.iloc[i:end_idx])[:, 1]

print(f"   ✓ Previsões concluídas")
print(f"   ✓ Probabilidade média: {y_pred_prob.mean():.4f}")
print(f"   ✓ Probabilidade mín: {y_pred_prob.min():.4f}")
print(f"   ✓ Probabilidade máx: {y_pred_prob.max():.4f}")

# === Criar array final com NaN onde faltou dado ===
print("\n Reconstruindo imagem espacial...")
pred_full = np.full(X.shape[0], np.nan, dtype="float32")
pred_full[~mask_nan] = y_pred_prob

# Voltar ao formato espacial (rows, cols)
mapa_pred = pred_full.reshape(rows, cols)

# === Atualizar perfil para 1 banda ===
profile.update(count=1, dtype="float32", nodata=np.nan)

# === Salvar GeoTIFF ===
print("\nSalvando mapa...")
with rasterio.open(output_map, "w", **profile) as dst:
    dst.write(mapa_pred, 1)

print(f"   ✓ Mapa salvo: {output_map}")

# === Estatísticas finais ===
print("\n" + "="*60)
print("ESTATÍSTICAS DO MAPA GERADO")
print("="*60)
print(f"Dimensões: {rows} x {cols} pixels")
print(f"Pixels classificados: {(~np.isnan(mapa_pred)).sum():,}")
print(f"Pixels com alta suscetibilidade (>0.7): {(mapa_pred > 0.7).sum():,}")
print(f"Pixels com média suscetibilidade (0.3-0.7): {((mapa_pred >= 0.3) & (mapa_pred <= 0.7)).sum():,}")
print(f"Pixels com baixa suscetibilidade (<0.3): {(mapa_pred < 0.3).sum():,}")
print("="*60)
print("✓ Processamento concluído com sucesso!")
print("="*60)